In [1]:
%pylab inline 
%load_ext autoreload
%autoreload 2
import numdifftools as nd
import tensorflow as tf
import numpy as np
import flowpm
import flowpm.tfpower as tfpower
import flowpm.scipy.interpolate as interpolate
from DifferentiableHOS.pk import pk as pkl
import pickle

Populating the interactive namespace from numpy and matplotlib


In [2]:
nsteps=10
nc=32
box_size=128
Omega_c=0.2589
sigma8=0.8159

In [3]:
def compute_initial_cond(Omega_c, sigma8):
    # Instantiates a cosmology with desired parameters
    cosmology = flowpm.cosmology.Planck15(Omega_c=Omega_c, sigma8=sigma8)
    stages = np.linspace(0.1, 1., nsteps, endpoint=True)
    # Compute linear matter power spectrum
    k = tf.constant(np.logspace(-4, 1, 256), dtype=tf.float32)
    pk = tfpower.linear_matter_power(cosmology, k)
    pk_fun = lambda x: tf.cast(tf.reshape(interpolate.interp_tf(tf.reshape(tf.cast(x, tf.float32), [-1]), k, pk), x.shape), tf.complex64)

    # And initial conditions
    initial_conditions = flowpm.linear_field([nc, nc, nc],
                                           [box_size, box_size,
                                           box_size],
                                           pk_fun,
                                           batch_size=1)
    return initial_conditions

In [17]:
initial_conditions=compute_initial_cond(Omega_c, sigma8).numpy()

In [18]:
@tf.function
def compute_Nbody_deriv(Omega_c, sigma8):
    """ Computes a N-body simulation for a given
    set of cosmological parameters
    """
    # Instantiates a cosmology with desired parameters
    cosmology = flowpm.cosmology.Planck15(Omega_c=Omega_c, sigma8=sigma8)
    stages = np.linspace(0.1, 1., nsteps, endpoint=True)
    # Compute linear matter power spectrum
#     k = tf.constant(np.logspace(-4, 1, 256), dtype=tf.float32)
#     pk = tfpower.linear_matter_power(cosmology, k)
#     pk_fun = lambda x: tf.cast(tf.reshape(interpolate.interp_tf(tf.reshape(tf.cast(x, tf.float32), [-1]), k, pk), x.shape), tf.complex64)

#     #And initial conditions
#     initial_conditions = flowpm.linear_field([nc, nc, nc],
#                                            [box_size, box_size,
#                                            box_size],
#                                            pk_fun,
#                                            batch_size=1)

    state = flowpm.lpt_init(cosmology, initial_conditions, 0.1)


    # Evolve particles from initial state down to a=af
    final_state = flowpm.nbody(cosmology, state, stages, [nc, nc,  nc])         

    # Retrieve final density field i.e interpolate the particles to the mesh
    final_field = flowpm.cic_paint(tf.zeros_like(initial_conditions), final_state[0])
    final_field=tf.reshape(final_field, [nc, nc, nc])
    params = tf.stack([Omega_c, sigma8])
    k, power_spectrum = pkl(final_field,shape=final_field.shape,boxsize=np.array([box_size, box_size,
                                               box_size]),kmin=0.1,dk=2*np.pi/box_size)
    k1 =tf.where(k < 0.5 ,False, True)
    k=tf.boolean_mask(k, tf.math.logical_not(k1))
    power_spectrum =tf.boolean_mask(power_spectrum, tf.math.logical_not(k1))
    return  power_spectrum

In [19]:
theoretical, numerical=tf.test.compute_gradient(
    compute_Nbody_deriv, [0.2589,0.8159], delta=0.01)

In [20]:
theoretical

(array([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], dtype=float32),
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32))

In [13]:
@tf.function
def compute_Nbody(Omega_c, sigma8):
  """ Computes a N-body simulation for a given
  set of cosmological parameters
  """
  # Instantiates a cosmology with desired parameters
  cosmology = flowpm.cosmology.Planck15(Omega_c=Omega_c, sigma8=sigma8)
  stages = np.linspace(0.1, 1., nsteps, endpoint=True)
  # Compute linear matter power spectrum
#   k = tf.constant(np.logspace(-4, 1, 256), dtype=tf.float32)
#   pk = tfpower.linear_matter_power(cosmology, k)
#   pk_fun = lambda x: tf.cast(tf.reshape(interpolate.interp_tf(tf.reshape(tf.cast(x, tf.float32), [-1]), k, pk), x.shape), tf.complex64)

#   #And initial conditions
#   initial_conditions = flowpm.linear_field([nc, nc, nc],
#                                            [box_size, box_size,
#                                            box_size],
#                                            pk_fun,
#                                            batch_size=1)

  state = flowpm.lpt_init(cosmology, initial_conditions, 0.1)

  
   # Evolve particles from initial state down to a=af
  final_state = flowpm.nbody(cosmology, state, stages, [nc, nc,  nc])         

  # Retrieve final density field i.e interpolate the particles to the mesh
  final_field = flowpm.cic_paint(tf.zeros_like(initial_conditions), final_state[0])
  final_field=tf.reshape(final_field, [nc, nc, nc])
  return final_field


#%%
@tf.function
def compute_jacobian(Omega_c, sigma8):
    """ Function that actually computes the Jacobian of a given statistics
    """
    params = tf.stack([Omega_c, sigma8])
    with tf.GradientTape() as tape:
        tape.watch(params)
        final_field = compute_Nbody(params[0], params[1])
        k, power_spectrum = pkl(final_field,shape=final_field.shape,boxsize=np.array([box_size, box_size,
                                               box_size]),kmin=0.1,dk=2*np.pi/box_size)
        k1 =tf.where(k < 0.5 ,False, True)
        k=tf.boolean_mask(k, tf.math.logical_not(k1))
        power_spectrum =tf.boolean_mask(power_spectrum, tf.math.logical_not(k1))

    return final_field, tape.jacobian(power_spectrum, params,experimental_use_pfor=False), k, power_spectrum



def main(Omega_c, sigma8):
  # Query the jacobian

    final_field, jacobian, k, power_spectrum = compute_jacobian(tf.convert_to_tensor(Omega_c,
                                                              dtype=tf.float32),
                                          tf.convert_to_tensor(sigma8,
                                                              dtype=tf.float32))
    
 
  # Saving results in requested filename
    return final_field, jacobian, k, power_spectrum

In [14]:
final_field, jacobian, k, power_spectrum=main(0.2589,0.8159)

In [ ]:
plot(k, numerical[1],label=r'${d P_k}/{d \sigma_8}$ num')
plot(k, jacobian[:,1],label=r'${d P_k}/{d \sigma_8}$ Flowpm')
plot(k, theoretical[1],label=r'${d P_k}/{d \sigma_8}$ theoretical ')
xlabel('k [Mpc]')
ylabel('$dP_k/P_k$')
legend()

In [ ]:
plot(k, numerical[0], label=r'${d P_k}/{d \Omega_c}$ num')
plot(k, jacobian[:,0],label=r'${d P_k}/{d \Omega_c}$ Flowpm')
plot(k, theoretical[0],label=r'${d P_k}/{d \Omega_c}$ theoretical')
xlabel('k [Mpc]')
ylabel('$dP_k$')
legend()